## Multilabel Classification

In [1]:
from robingrad import Tensor, draw_dot
import robingrad.nn as nn
from robingrad.optim import SGD
from robingrad.state import get_parameters
from sklearn import datasets
import pandas as pd
import numpy as np
import edamame.eda as eda
import time
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

### Dataset

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
print(X.shape)
print(type(X))
print(y.shape)
print(type(y))
num_classes = len(np.unique(y))
print(num_classes)

(150, 4)
<class 'numpy.ndarray'>
(150,)
<class 'numpy.ndarray'>
3


In [3]:
X = pd.DataFrame(X, columns=iris.feature_names)
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
y = pd.Series(y)
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [5]:
X_train, y_train, X_test, y_test = eda.setup(X,y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [6]:
X_train_s = eda.scaling(X_train)
X_test_s = eda.scaling(X_test)
y_train = y_train.squeeze().to_numpy()
y_test = y_test.squeeze().to_numpy()
print(X_train_s.shape)
print(X_test_s.shape)
print(y_train.shape)
print(y_test.shape)

(112, 4)
(38, 4)
(112,)
(38,)


In [7]:
y_train_ohe = np.eye(num_classes)[y_train]
y_train_ohe[:5]

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [8]:
X_train_t = Tensor(X_train_s, requires_grad=True)
print(type(X_train_t), X_train_t.shape)
y_train_ohe_t = Tensor(y_train_ohe, requires_grad=True).reshape((112,3))
print(type(y_train_ohe_t), y_train_ohe_t.shape)

<class 'robingrad.tensor.Tensor'> (112, 4)
<class 'robingrad.tensor.Tensor'> (112, 3)


In [93]:
class RobinNet:
    def __init__(self):
        self.l1 = nn.Linear(4, 16, mean=0.001)
        self.l2 = nn.Linear(16, 16, mean=0.001)
        self.l3 = nn.Linear(16, 3, mean=0.001)
    def __call__(self, x):
        x = self.l1(x)
        x = x.relu()
        x = self.l2(x)
        x = x.relu()
        x = self.l3(x)

        return x

In [94]:
model = RobinNet()
params = get_parameters(model)
opt = SGD(params)

In [95]:
def cross_entropy_loss(forward_array, target):
    exp_forward = forward_array.exp()
    norm_term = Tensor.full_like(exp_forward, fill_value=exp_forward.sum().data, requires_grad=True)
    softmax_vec = exp_forward/norm_term
    neg_output = 1 - softmax_vec
    bce_losses = -(target * softmax_vec.log() + (1-target)*neg_output.log())
    BCELoss = bce_losses.sum()
    y_pred = softmax_vec.data.tolist()[0].index(max(softmax_vec.data.tolist()[0]))
    return BCELoss, y_pred

In [96]:
def accuracy_val(y_pred, y_true):
    y_pred = np.array(y_pred).round()
    correct_results = np.sum(y_pred == y_true)
    acc = correct_results/y_true.shape[0]
    acc = np.round(acc * 100)
    return acc

In [97]:
epochs = 200
for epoch in range(epochs):
    y_pred = []
    losses = []
    s = time.monotonic()
    for i in range(X_train_t.shape[0]):
        output = model(X_train_t[i].reshape((1,4)))
        loss, y_pred_i = cross_entropy_loss(output, y_train_ohe_t[i].reshape((1,3)))
        losses.append(loss.data)
        y_pred.append(y_pred_i)
        opt.zero_grad()
        loss.backward()
        opt.step()
    e = time.monotonic()
    t = e - s
    acc = accuracy_val(y_pred=y_pred, y_true=y_train)
    loss_epoch = sum(losses)/len(losses)
    if epoch % 10 == 0 or epoch==(epochs-1):
        print(f"epoch: {epoch} | loss: {loss_epoch:.2f} | time: {t:.2f} sec. | acc: {acc:.2f}%")

ValueError: can't log negative or zero value

In [98]:
model = RobinNet()
params = get_parameters(model)
opt = SGD(params)

In [115]:
index = 19
output = model(X_train_t[index].reshape((1,4)))
print(f"output: {output.data}")
target = y_train_ohe_t[index].reshape((1,3))
print(f"target: {target.data}")
y_true = y_train[index]
print(f"y_true: {y_true}")
def cross_entropy_loss(forward_array, target):
    exp_forward = forward_array.exp()
    print(f"exp_forward: {exp_forward.data}")
    norm_term = Tensor.full_like(exp_forward, fill_value=exp_forward.sum().data, requires_grad=True)
    print(f"norm_term: {norm_term.data}")
    softmax_vec = exp_forward/norm_term
    print(f"softmax_vec: {softmax_vec.data}")
    neg_output = 1 - softmax_vec
    print(f"neg_output: {neg_output.data}")
    bce_losses = -(target * softmax_vec.log() + (1-target)*neg_output.log())
    print(f"bce_losses: {bce_losses.data}")
    BCELoss = bce_losses.sum()
    print(f"BCELoss: {BCELoss.data}")
    y_pred = softmax_vec.data.tolist()[0].index(max(softmax_vec.data.tolist()[0]))
    return BCELoss, y_pred
loss, pred = cross_entropy_loss(output, target)
opt.zero_grad()
loss.backward()
opt.step()
print(pred)

output: [[ 13.452124  -15.72418    -3.9871705]]
target: [[1. 0. 0.]]
y_true: 0
exp_forward: [[6.9531731e+05 1.4827752e-07 1.8552134e-02]]
norm_term: [[695317.3 695317.3 695317.3]]
softmax_vec: [[1.0000000e+00 2.1325159e-13 2.6681535e-08]]
neg_output: [[0. 1. 1.]]


ValueError: can't log negative or zero value

In [124]:
np.log(1e-20)

-46.051701859880914

In [122]:
1e-10

1e-10